In [1]:
!pip install pandas
!pip install python-dotenv
!pip install openai
!pip install tiktoken
!pip install numpy
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
from dotenv import load_dotenv
import openai
from openai import OpenAI
import tiktoken
from ast import literal_eval
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [3]:
# https://wortschatz.uni-leipzig.de/en/download/Hungarian
df = pd.read_csv('hun_news_2023_10K-sentences.txt', sep='\t', index_col=0, names=['sentence'], header=None, skipinitialspace=True)
df.head()

,sentence
1,A 11. hónapban felvett lakáshitelek átlagos ka...
2,A 11 százalékos feldolgozóipari súlyú villamos...
3,"A 133-as és a 130-as km között, a főváros felé..."
4,A 136. dandár orosz katonái megsemmisítettek e...
5,A 138-as és a 144-es kilométer között lezárták...


In [4]:
dataset = df.sample(500, random_state=42)
dataset.head()

,sentence
4336,Az ügyben a nyomozás még folyamatban van.
7262,"Lapunk is rákérdezett erre a tárcánál, de nem ..."
5003,Egyébként arról is lehetne valami info hogy ez...
222,A bölcsődei ellátás július 17-től 21-ig; valam...
9336,"Tavaly 1,3 millió forintnyi állampapírt jegyze..."


In [5]:
load_dotenv()
MODEL = 'gpt-3.5-turbo'
CLIENT = OpenAI()

In [6]:
CLIENT.chat.completions.create(model=MODEL, messages=[{'role': 'system', 'content': 'You are a helpful chatbot.'},
                                                         {'role': 'user', 'content': 'Helló!'}]).choices[0].message.content

'Hello! How can I assist you today?'

In [7]:
def request_completion(sys_msg, usr_msg, logit_bias):
    completion_response = CLIENT.chat.completions.create(
                            messages=[
                                {'role': 'system', 'content': sys_msg},
                                {'role': 'user', 'content': usr_msg}
                            ],
                            temperature=0,
                            max_tokens=1,
                            top_p=1,
                            frequency_penalty=0,
                            presence_penalty=0,
                            model=MODEL,
                            logit_bias=logit_bias)

    return completion_response

def classify_sentence(sentence, categories, logit_bias):
    sys_msg = """You are a language expert working for a hungarian news organisation.
You are analysing sentences from hungarian news articles and classifying them into the following categories:
politics, trafic, technology, economy, other.
Your aswer should be the id of the correct category according to the following table:
ID,Category
""" + '\n'.join([f'{idx},{category}' for idx, category in enumerate(categories)])
    usr_msg = "The sentence is: " + sentence
    
    return categories[int(request_completion(sys_msg, usr_msg, logit_bias)
                                             .choices[0].message.content)]

def generate_logit_bias(ids):
    encoding = tiktoken.encoding_for_model(MODEL)
    token_ids = list(map(lambda x: encoding.encode(x)[0], ids))
    return {token_id: 100 for token_id in token_ids}
        
    

In [8]:
categories = ['politics', 'trafic', 'technology', 'economy', 'music', 'holidays', 'sport', 'other']
category_logit_bias = generate_logit_bias([f'{idx}' for idx, _ in enumerate(categories)])

test_dataset = dataset.iloc[:10].copy()

test_dataset['topic'] = test_dataset.apply(lambda x: classify_sentence(x['sentence'], categories, category_logit_bias), axis=1)

test_dataset


,sentence,topic
4336,Az ügyben a nyomozás még folyamatban van.,other
7262,"Lapunk is rákérdezett erre a tárcánál, de nem ...",politics
5003,Egyébként arról is lehetne valami info hogy ez...,technology
222,A bölcsődei ellátás július 17-től 21-ig; valam...,other
9336,"Tavaly 1,3 millió forintnyi állampapírt jegyze...",economy
4301,"Azt is tudjuk, hogy akik már voltak, kivétel n...",other
9965,"Vörösréz lemezzel bélelt, nagyjából ellipszis ...",other
6337,"Hangsúlyozni szeretném, hogy eddig is ebbe az ...",politics
1540,A lap felidézte a Eurostat 2021-es évre vonatk...,economy
8752,"Pétert, ahogy azt sem találta megalapozottnak,...",politics


In [9]:
dataset['topic'] = dataset.apply(lambda x: classify_sentence(x['sentence'], categories, category_logit_bias), axis=1)
dataset.to_csv('labeled_sentences.csv')

In [10]:
embedding_model = 'text-embedding-3-small'

In [11]:
dataset = pd.read_csv('labeled_sentences.csv', index_col=0)
dataset.head()

,sentence,topic
4336,Az ügyben a nyomozás még folyamatban van.,other
7262,"Lapunk is rákérdezett erre a tárcánál, de nem ...",politics
5003,Egyébként arról is lehetne valami info hogy ez...,technology
222,A bölcsődei ellátás július 17-től 21-ig; valam...,other
9336,"Tavaly 1,3 millió forintnyi állampapírt jegyze...",economy


In [12]:
dataset['embedding'] = dataset.apply(lambda x: client.embeddings.create(input=[x['sentence']], model=embedding_model)
                                     .data[0].embedding, axis=1)
dataset

,sentence,topic,embedding
4336,Az ügyben a nyomozás még folyamatban van.,other,"[-0.024209091439843178, 0.0666579082608223, -0..."
7262,"Lapunk is rákérdezett erre a tárcánál, de nem ...",politics,"[-0.062187161296606064, 0.029136642813682556, ..."
5003,Egyébként arról is lehetne valami info hogy ez...,technology,"[-0.009416894987225533, 0.011644127778708935, ..."
222,A bölcsődei ellátás július 17-től 21-ig; valam...,other,"[-0.024719061329960823, 0.02838403359055519, 0..."
9336,"Tavaly 1,3 millió forintnyi állampapírt jegyze...",economy,"[-0.014268632978200912, 0.017591465264558792, ..."
...,...,...,...
1668,A logisztika világa ezeknek a lapoknak a méret...,trafic,"[-0.012719177640974522, -0.00961320474743843, ..."
4185,Az összes elektronikus tranzakció típus emelke...,economy,"[-0.015369594097137451, -0.0019071587594226003..."
767,"A H5-ös, a H6-os, a H8-as és a H9-es HÉV vonal...",trafic,"[-0.05273379758000374, 0.038919560611248016, -..."
3254,"A vitorlázás esetében azonban tudni kell, hogy...",other,"[-0.020395955070853233, 0.023480020463466644, ..."


In [13]:
dataset.to_csv('labeled_embedded_sentences.csv')

In [14]:
dataset = pd.read_csv('labeled_embedded_sentences.csv', index_col=0)
dataset['embedding'] = dataset.embedding.apply(literal_eval).apply(np.array)
dataset.head()

,sentence,topic,embedding
4336,Az ügyben a nyomozás még folyamatban van.,other,"[-0.024209091439843178, 0.0666579082608223, -0..."
7262,"Lapunk is rákérdezett erre a tárcánál, de nem ...",politics,"[-0.062187161296606064, 0.029136642813682556, ..."
5003,Egyébként arról is lehetne valami info hogy ez...,technology,"[-0.009416894987225533, 0.011644127778708935, ..."
222,A bölcsődei ellátás július 17-től 21-ig; valam...,other,"[-0.024719061329960823, 0.02838403359055519, 0..."
9336,"Tavaly 1,3 millió forintnyi állampapírt jegyze...",economy,"[-0.014268632978200912, 0.017591465264558792, ..."


In [16]:
X_train, X_eval, y_train, y_eval = train_test_split(list(dataset.embedding.values), 
                                                   list(dataset.topic.values), 
                                                   test_size=.2, random_state=42)
rnd_forest = RandomForestClassifier(n_estimators=100)
rnd_forest.fit(X_train, y_train)

RandomForestClassifier()

In [19]:
preds = rnd_forest.predict(X_eval)
probs = rnd_forest.predict_proba(X_eval)

report = classification_report(y_eval, preds, zero_division=np.nan)
print(report)

              precision    recall  f1-score   support

     economy       1.00      0.62      0.77        16
    holidays       0.00      0.00      0.00         1
       music       0.00      0.00      0.00         1
       other       0.61      0.98      0.76        52
    politics       0.86      0.27      0.41        22
       sport       0.00      0.00      0.00         2
  technology       0.00      0.00      0.00         4
      trafic       0.00      0.00      0.00         2

    accuracy                           0.67       100
   macro avg       0.31      0.23      0.24       100
weighted avg       0.67      0.67      0.61       100



/home/aszfalt/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/aszfalt/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/aszfalt/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [20]:
mlp = MLPClassifier(hidden_layer_sizes=(1000,500,250), random_state=42, max_iter=500)
mlp.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(1000, 500, 250), max_iter=500,
              random_state=42)

In [22]:
preds = mlp.predict(X_eval)
probs = mlp.predict_proba(X_eval)

report = classification_report(y_eval, preds, zero_division=np.nan)
print(report)

              precision    recall  f1-score   support

     economy       0.68      0.81      0.74        16
    holidays       0.00      0.00      0.00         1
       music       0.00      0.00      0.00         1
       other       0.84      0.83      0.83        52
    politics       0.82      0.82      0.82        22
       sport       0.33      1.00      0.50         2
  technology       0.00      0.00      0.00         4
      trafic       0.50      0.50      0.50         2

    accuracy                           0.77       100
   macro avg       0.40      0.49      0.42       100
weighted avg       0.74      0.77      0.75       100



/home/aszfalt/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/aszfalt/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/aszfalt/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
